In [16]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
drive.mount('/content/drive')
from sklearn.model_selection import GroupKFold
from sklearn.model_selection import train_test_split


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
def read_data(train_csv,val_csv,test_csv):
  train_df = pd.read_csv(train_csv)
  test_df = pd.read_csv(test_csv)
  val_df = pd.read_csv(val_csv)
  return train_df, val_df, test_df

train_val_path = '/content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/imputed_values/'
test_path = '/content/drive/MyDrive/WIDS-Datathon-2024/Data/Test/'


train_csv = train_val_path + 'train_df.csv'
val_csv = train_val_path + 'val_df.csv'
test_csv = test_path + 'test.csv'
train_df, val_df, test_df = read_data(train_csv, val_csv, test_csv)
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)

(10324, 82)
(2582, 83)
(5792, 82)


In [3]:
columns_diff = set(train_df.columns) - set(val_df.columns)
columns_diff

extra_column_in_val = set(val_df.columns) - set(train_df.columns)

print(extra_column_in_val)


{'predicted_patient_state'}


In [4]:
val_df.drop('predicted_patient_state',  axis=1, inplace=True)

In [24]:
path = '/content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/'
train_df_encoded = pd.read_csv(path + 'train_df_encoded.csv')
print(train_df_encoded.shape)

(13071, 877)


In [12]:
duplicate_patient_ids = train_df_encoded[train_df_encoded.duplicated(subset=['patient_id'], keep=False)]
duplicate_patient_ids

,patient_id,patient_age,income_household_25_to_35,rent_burden,income_household_under_5,DiagPeriodL90D,breast_cancer_diagnosis_code_1741,breast_cancer_diagnosis_code_1742,breast_cancer_diagnosis_code_1743,breast_cancer_diagnosis_code_1744,...,patient_zip3_986,patient_zip3_988,patient_zip3_989,patient_zip3_990,patient_zip3_991,patient_zip3_992,patient_zip3_993,patient_zip3_994,patient_zip3_996,patient_zip3_998


In [41]:
import pandas as pd
import os

def create_and_save_folds(df, path):
    """
    Creates and saves cross-validation folds with specified sizes and no overlap.

    Args:
    - df (pd.DataFrame): The DataFrame containing the data.
    - path (str): The base path where the fold files will be saved.

    Returns:
    None
    """
    df_shuffled = df.sample(frac=1, random_state=42).reset_index(drop=True)

    val_fold_size = 653  # Size of the validation fold
    total_val_records = 4 * val_fold_size

    # Use a set to keep track of used indices
    used_indices = set()

    for fold_number in range(4):
        # Determine the start and end indices for the validation fold
        start_index_val = fold_number * val_fold_size
        end_index_val = start_index_val + val_fold_size

        # Extract indices for the current validation fold
        val_fold_indices = list(range(start_index_val, end_index_val))
        val_fold_indices = [i for i in val_fold_indices if i not in used_indices]  # Remove already used indices

        # Extract indices for the training fold
        train_indices = set(range(len(df_shuffled))) - set(val_fold_indices)
        train_indices = list(train_indices)[:2614]  # Limit train fold size to 2614

        # Update used indices with the current validation fold indices
        used_indices.update(val_fold_indices)

        # Extract the data for the folds
        val_fold = df_shuffled.iloc[val_fold_indices]
        train_fold = df_shuffled.iloc[train_indices]

        # Check for duplicates in each fold
        if val_fold.duplicated().any() or train_fold.duplicated().any():
            print(f"Duplicate data points found in Fold {fold_number+1}. Please review your sampling strategy.")

        # Save the folds to CSV files
        train_fold_filename = os.path.join(path, f'train_fold_{fold_number+1}.csv')
        val_fold_filename = os.path.join(path, f'val_fold_{fold_number+1}.csv')

        train_fold.to_csv(train_fold_filename, index=False)
        val_fold.to_csv(val_fold_filename, index=False)

        print(f'Fold {fold_number+1} written to {train_fold_filename} and {val_fold_filename}')

create_and_save_folds(train_df_encoded, path)

Fold 1 written to /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/train_fold_1.csv and /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/val_fold_1.csv
Fold 2 written to /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/train_fold_2.csv and /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/val_fold_2.csv
Fold 3 written to /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/train_fold_3.csv and /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/val_fold_3.csv
Fold 4 written to /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/train_fold_4.csv and /content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/val_fold_4.csv


In [43]:
def read_and_print_fold_shapes(path, num_folds=4):
    for fold_number in range(1, num_folds + 1):
        train_fold_filename = path + f'train_fold_{fold_number}.csv'
        val_fold_filename = path + f'val_fold_{fold_number}.csv'

        train_fold = pd.read_csv(train_fold_filename)
        val_fold = pd.read_csv(val_fold_filename)

        print(f'Fold {fold_number} Training Set Shape: {train_fold.shape}')
        print(f'Fold {fold_number} Validation Set Shape: {val_fold.shape}')

# Specify the path where the folds are stored
path = '/content/drive/MyDrive/WIDS-Datathon-2024/Hamsini/cross_val/'

read_and_print_fold_shapes(path)


Fold 1 Training Set Shape: (2614, 877)
Fold 1 Validation Set Shape: (653, 877)
Fold 2 Training Set Shape: (2614, 877)
Fold 2 Validation Set Shape: (653, 877)
Fold 3 Training Set Shape: (2614, 877)
Fold 3 Validation Set Shape: (653, 877)
Fold 4 Training Set Shape: (2614, 877)
Fold 4 Validation Set Shape: (653, 877)
